In [6]:

# Importing the PIL library
from PIL import Image
from PIL import ImageDraw, ImageFont

# Importing the XML library
from xml.dom.minidom import parse

import re
from copy import deepcopy

# VKBSim Gladiator Pro Layout Generator
## Description
This is a python script to generate a visual layout for an exported button configuration for the game Star Citizen.
This is not an official tool and has not been approved by Cloud Imperium Games or Roberts Space Industries.

## How it works
The script uses a base image (taken from https://imgur.com/a/vkb-gladiator-nxt-premium-binding-sheets-9IWuSYp) and an exported layout file in xml format.
The information in that layout file are then parsed and mapped to pre-defined positions in the image.
The final result is then exported as your layout file.

## Configuration
TODO
- Switch left/right
- Define custom replacement text in a file
- Which parts to export and which to merge

In [7]:
STANDARD_ANCHOR = "md"
FILL = "black"
POSITION = "pos"
WIDTH = "width"
KEYBIND = "keybind"
FONTTYPE = "arial.ttf"
FONTSIZE = "font_size"
DEFAULT_FIRE_ACTION = "Fire (Group1)"

JS_REGEX = r"js\d_\S"

# (a,b,c): a = left, b = right, c = bottom
JOYSTICK_DATA = {
    # BUTTONS
    "js1_button1" : {POSITION: (1481, 1561, 288), KEYBIND: DEFAULT_FIRE_ACTION},
    "js1_button2" : {POSITION: (1481, 1561, 305), KEYBIND: None},
    "js1_button3" : {POSITION: (902, 981, 254), KEYBIND: None},
    "js1_button4" : {POSITION: (1462, 1540, 203), KEYBIND: None},
    "js1_button5" : {POSITION: (1462, 1540, 339), KEYBIND: None},
    "js1_button6" : {POSITION: (1142, 1221, 118), KEYBIND: None},
    "js1_button7" : {POSITION: (1242, 1321, 135), KEYBIND: None},
    "js1_button8" : {POSITION: (1142, 1221, 152), KEYBIND: None},
    "js1_button9" : {POSITION: (1042, 1121, 135), KEYBIND: None},
    "js1_button10" : {POSITION: (1142, 1221, 135), KEYBIND: None},
    "js1_button11" : {POSITION: (1302, 1381, 50), KEYBIND: None},
    "js1_button12" : {POSITION: (1402, 1481, 67), KEYBIND: None},
    "js1_button13" : {POSITION: (1302, 1381, 84), KEYBIND: None},
    "js1_button14" : {POSITION: (1202, 1281, 67), KEYBIND: None},
    "js1_button15" : {POSITION: (1302, 1381, 67), KEYBIND: None},
    "js1_button16" : {POSITION: (802, 881, 388), KEYBIND: None},
    "js1_button17" : {POSITION: (902, 981, 405), KEYBIND: None},
    "js1_button18" : {POSITION: (802, 881, 422), KEYBIND: None},
    "js1_button19" : {POSITION: (702, 781, 405), KEYBIND: None},
    "js1_button20" : {POSITION: (802, 881, 405), KEYBIND: None},
    "js1_button21" : {POSITION: (1502, 1581, 237), KEYBIND: None},
    "js1_button22" : {POSITION: (1502, 1581, 254), KEYBIND: None},
    "js1_button23" : {POSITION: (861, 941, 592), KEYBIND: None},
    "js1_button24" : {POSITION: (861, 941, 609), KEYBIND: None},
    "js1_button25" : {POSITION: (1502, 1581, 526), KEYBIND: None},
    "js1_button26" : {POSITION: (1502, 1581, 543), KEYBIND: None},
    "js1_button27" : {POSITION: (1522, 1601, 458), KEYBIND: None},
    "js1_button28" : {POSITION: (1482, 1561, 475), KEYBIND: None},
    "js1_button29" : {POSITION: (1562, 1641, 441), KEYBIND: None},
    "js2_button1" : {POSITION: (121, 200, 288), KEYBIND: None},
    "js2_button2" : {POSITION: (121, 200, 305), KEYBIND: None},
    "js2_button3" : {POSITION: (701, 780, 254), KEYBIND: None},
    "js2_button4" : {POSITION: (141, 220, 203), KEYBIND: None},
    "js2_button5" : {POSITION: (141, 220, 339), KEYBIND: None},
    "js2_button6" : {POSITION: (461, 540, 118), KEYBIND: None},
    "js2_button7" : {POSITION: (561, 640, 135), KEYBIND: None},
    "js2_button8" : {POSITION: (461, 540, 152), KEYBIND: None},
    "js2_button9" : {POSITION: (361, 440, 135), KEYBIND: None},
    "js2_button10" : {POSITION: (461, 540, 135), KEYBIND: None},
    "js2_button11" : {POSITION: (301, 380, 50), KEYBIND: None},
    "js2_button12" : {POSITION: (401, 480, 67), KEYBIND: None},
    "js2_button13" : {POSITION: (301, 380, 84), KEYBIND: None},
    "js2_button14" : {POSITION: (201, 280, 67), KEYBIND: None},
    "js2_button15" : {POSITION: (301, 380, 67), KEYBIND: None},
    "js2_button16" : {POSITION: (801, 880, 322), KEYBIND: None},
    "js2_button17" : {POSITION: (901, 980, 339), KEYBIND: None},
    "js2_button18" : {POSITION: (801, 880, 356), KEYBIND: None},
    "js2_button19" : {POSITION: (701, 780, 339), KEYBIND: None},
    "js2_button20" : {POSITION: (801, 880, 339), KEYBIND: None},
    "js2_button21" : {POSITION: (101, 180, 237), KEYBIND: None},
    "js2_button22" : {POSITION: (101, 180, 254), KEYBIND: None},
    "js2_button23" : {POSITION: (121, 200, 526), KEYBIND: None},
    "js2_button24" : {POSITION: (121, 200, 543), KEYBIND: None},
    "js2_button25" : {POSITION: (741, 820, 526), KEYBIND: None},
    "js2_button26" : {POSITION: (741, 820, 543), KEYBIND: None},
    "js2_button27" : {POSITION: (81, 160, 441), KEYBIND: None},
    "js2_button28" : {POSITION: (41, 120, 424), KEYBIND: None},
    "js2_button29" : {POSITION: (121, 200, 458), KEYBIND: None},
    # AXES
    "js1_x" : {POSITION: (1162, 1261, 645), KEYBIND: "Pitch/Yaw"},
    "js1_y" : {POSITION: (1162, 1261, 645), KEYBIND: "Pitch/Yaw"},
    "js1_z" : {POSITION: (1302, 1401, 645), KEYBIND: "Roll"},
    "js1_hat1_up" : {POSITION: (982, 1061, 50), KEYBIND: None},
    "js1_hat1_down" : {POSITION: (982, 1061, 84), KEYBIND: None},
    "js1_hat1_left" : {POSITION: (882, 961, 67), KEYBIND: None},
    "js1_hat1_right" : {POSITION: (1082, 1161, 67), KEYBIND: None},
    "js1_rotx" : {POSITION: (0,0,0), KEYBIND: None},
    "js1_roty" : {POSITION: (0,0,0), KEYBIND: None},
    "js1_rotz" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_x" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_y" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_z" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_hat1_up" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_hat1_down" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_hat1_left" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_hat1_right" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_rotx" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_roty" : {POSITION: (0,0,0), KEYBIND: None},
    "js2_rotz" : {POSITION: (0,0,0), KEYBIND: None}
}


COMBINED_CATEGORIES = {
    "Flight/NAV" : r"spaceship_(general|view|movement|quantum|scanning)|seat_.+|lights_.+",
    "Combat (Weapons)" : r"spaceship_(general|view|movement|weapons|defensive)|seat_.+|lights_.+",
    "Combat (Missiles)" : r"spaceship_(general|view|movement|missiles|defensive)|seat_.+|lights_.+",
    "Mining" : r"spaceship_(general|view|movement|mining)|seat_.+|lights_.+",
    "Salvage" : r"spaceship_(general|view|movement|salvage)|seat_.+|lights_.+",
    "Radar" : r"spaceship_(general|view|movement|radar|defensive)|seat_.+|lights_.+"
}

 


In [8]:
class joysticks:
    is_reversed = False
    MAX_FONT_SIZE = 12
    TITLE_FONT_SIZE = 28
    NAME_REPLACEMENTS = {
        "Weapon" : "",
        "Attack" : "Weapons",
        "Mining" : "", 
        "Salvage" : "",
        "Atc" : "ATC", 
        "Countermeasure" : "CM",
        "Vtol" : "VTOL", 
        "Scanning Trigger " : "",
        "Toggle Yaw Roll Swap" : "Swap Yaw Roll",
        "Operator Mode" : "Op Mode",
        "Master Mode" : "MM", 
        "Dec " : "- ",
        "Inc " : "+ ",
        "Ifcs " : "",
        "Toggle Vector Decoupling": "Toggle Decoupled",
        "Throttle Swap Mode" : "Toggle Sticky Throttle",
        "Landing System" : "Landing Gear",
        "Reset Scm" : "Reset",
        "Cycle Forward" : "Up",
        "Cycle Back" : "Down",
        "Toggle Qdrive Engagement" : "Engage Q-Drive",
        "Throttle Set Normal" : "Throttle Reset",
        "Increase" : "+",
        "Decrease" : "-",
        "Cycle Missile Fwd" : "Next Missile",
        "Cycle Missile Back" : "Prev Missile",
        "CM Decoy Launch" : "Launch Decoy",
        "CM Noise Launch" : "Launch Noise",
        " Lead Lag" : "",
        "Stable Max Zoom Hold" : "Max Zoom",
        "Fire Fracture" : "Fracture",
        "Fire Disintegrate" : "Disintegrate"
    }
    PADDING_BOT = 1
    PADDING_SIDES = 1

    def __init__(self, categories, is_reversed):
        self.data = {k: deepcopy(JOYSTICK_DATA) for k in categories}
        self._update_position_data(categories)
        self.is_reversed = is_reversed
        for cat in self.data:
            for key in self.data[cat]:
                self.data[cat][key][FONTSIZE] = self.MAX_FONT_SIZE

    def _update_position_data(self, categories):
        for k in categories:
            category_data = self.data[k]
            for key in category_data:
                width = self._get_width(category_data[key][POSITION])
                position = self._get_pos(category_data[key][POSITION])
                category_data[key][POSITION] = position
                category_data[key][WIDTH] = width

    def _get_pos(self, entry):
        return (int(entry[0] + self._get_width(entry) / 2), entry[2] - self.PADDING_BOT)
    
    def _get_width(self, entry):
        return entry[1] - entry[0]
    
    def _determine_text_size(self, text, width):
        return 
    
    def _reverse_key(self, key):
        if "js1_" in key:
            return key.replace("js1_","js2_")
        else:
            return key.replace("js2_","js1_")
        
    def _transform_name(self, name):
        new_name = name.replace("v_","").replace("_"," ").title()
        for long_name in self.NAME_REPLACEMENTS.keys():
            new_name = new_name.replace(long_name, self.NAME_REPLACEMENTS[long_name])
        return new_name
    
    def _determine_font_size(self, category, key, draw):
        font_size = self.MAX_FONT_SIZE
        available_width = self.data[category][key][WIDTH] - self.PADDING_SIDES
        text = self.data[category][key][KEYBIND].split("\n")

        while True:
            font = ImageFont.truetype(FONTTYPE, font_size)
            current_length = max([draw.textlength(x, font=font) for x in text])
            if current_length <= available_width or available_width <= 0:
                break
            font_size = font_size - 1

        return font
    
    def add_keybind(self, category, key, name):
        actual_key = self._reverse_key(key) if self.is_reversed else key
        if self.data[category][actual_key][KEYBIND] and not (actual_key == "js1_button1" and self.data[category][actual_key][KEYBIND] == DEFAULT_FIRE_ACTION):
            print(f"WARNING: Duplicate entry for {KEYBIND}: {self.data[category][actual_key][KEYBIND]} - {self._transform_name(name)}")
            self.data[category][actual_key][KEYBIND] = f"{self.data[category][actual_key][KEYBIND]} |\n{self._transform_name(name)}"

        else:
            self.data[category][actual_key][KEYBIND] = self._transform_name(name)

    def draw_keybinds(self, category, draw):
        for key in self.data[category]:
            if self.data[category][key][KEYBIND]:
                if self.data[category][key][POSITION] == (0,0):
                    print(f"WARNING: Undefined position for {key} - {self.data[category][key][KEYBIND]}")
                font = self._determine_font_size(category, key, draw)
                draw.text(self.data[category][key][POSITION], self.data[category][key][KEYBIND], fill=FILL, anchor=STANDARD_ANCHOR, font=font)

        title_font = ImageFont.truetype(FONTTYPE, self.TITLE_FONT_SIZE)
        draw.text((10,10), category, fill=FILL, anchor="la", font=title_font)
        
        
def determine_joystick_order(keybinds_xml):
    reverse_order = False
    joysticks = [x for x in keybinds_xml.getElementsByTagName("options") if x.getAttribute("type") == "joystick"] if keybinds_xml and keybinds_xml.getElementsByTagName("options") else None
    if joysticks:
        joystick_left = [x for x in joysticks if " L " in x.getAttribute("Product")][0]
        reverse_order = True if joystick_left.getAttribute("instance") == "1" else reverse_order
    return reverse_order



In [9]:
keybinds_xml = ""
with open("layout.xml", "r") as file:
    keybinds_xml = parse(file)

reverse_order = determine_joystick_order(keybinds_xml)

In [10]:
action_maps = keybinds_xml.getElementsByTagName("actionmap")
categories = COMBINED_CATEGORIES.keys()

my_joysticks = joysticks(categories, reverse_order)

for category in categories:
    layout = Image.open("res/vkbsim_default_layout.png")
    draw = ImageDraw.Draw(layout)
    relevant_action_maps = [a for a in action_maps if re.search(COMBINED_CATEGORIES[category], a.getAttribute("name"))]
    for action_map in relevant_action_maps:
        for action in action_map.getElementsByTagName("action"):
            rebind_list = [x.getAttribute("input") for x in action.getElementsByTagName("rebind") if re.search(JS_REGEX, x.getAttribute("input"))]
            if rebind_list:
                my_joysticks.add_keybind(category, rebind_list[0], action.getAttribute("name"))

    my_joysticks.draw_keybinds(category, draw)
    name = category.replace("/","-")
    layout.save(f"Keybindings_{name}.png")
    # layout.show()
    